In [32]:
import os
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import scipy.stats as stats

# --- Statistical tests ---
from scipy.stats import kruskal, shapiro
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols


# --- Post-hoc comparisons (optional) ---
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
proj_dir="/master/nplatt/sch_hae_its-nigeria"
results_dir=f"{proj_dir}/results"

In [8]:
#you can get this table from the SCAN dryad link
info_df=pd.read_csv(f"{proj_dir}/its-nigeria_samplesheet.csv", sep=",")

# Data wrangling/clean up

In [4]:
Path(f"{results_dir}/anova").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/anova")

In [25]:
# Read in the data
df=pd.read_csv(f"{results_dir}/combined/merged.csv", header=0, sep=",")

# Select target columns from Nigerian S. haematobium for ANOVA
cols = ["wgs_id", "locale", "its_class", "mitotype", "adm2"]
df = df.loc[ ((df["country"] == "nigeria") &
              (df["species"] == "shaematobium")), cols].reset_index(drop=True)
df = df[cols].dropna()

# Ensure data is represented as correct type.
for c in ["locale", "its_class", "mitotype"]:
    df[c] = df[c].astype("category")
    
assert np.issubdtype(df["adm2"].dtype, np.number)


,wgs_id,locale,its_class,mitotype,adm2
0,c_Sh_NG_bo_1_3,borno,SHxSH,SH,0.999980
1,c_Sh_NG_bo_3_1,borno,SHxSH,SH,0.999980
2,c_Sh_NG_bo_3_2,borno,SHxSH,SH,0.999980
3,c_Sh_NG_bo_4_1,borno,SHxSH,SH,0.999980
4,c_Sh_NG_bo_5_2,borno,SHxSH,SH,0.999980
...,...,...,...,...,...
127,c_Sh_NG_os_3_3,osun,SHxSH,SH,0.999980
128,Sh_NG_os_3_1,osun,SHxSH,SH,0.999980
129,c_Sh_NG_os_3_11,osun,SHxSH,SH,0.996946
130,c_Sh_NG_os_3_5,osun,SHxSH,SH,0.999980


In [46]:
from scipy.stats import shapiro

stat, p = shapiro(df["adm2"])
print(f"Shapiro-Wilk W={stat:.3f}, p={p:.3g}")

Shapiro-Wilk W=0.697, p=3.75e-15


In [44]:
def kruskal_by(col):
    groups = [g["adm2"].values for _, g in df.groupby(col)]
    if len(groups) < 2:
        return np.nan, np.nan
    stat, p = kruskal(*groups)
    return stat, p

kw_rows = []
for c in ["locale", "its_class", "mitotype"]:
    H, p = kruskal_by(c)
    kw_rows.append({"factor": c, "kruskal_H": H, "p_value": p})
kw = pd.DataFrame(kw_rows).sort_values("factor")
kw

/tmp/ipykernel_150077/2383160622.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = [g["adm2"].values for _, g in df.groupby(col)]


,factor,kruskal_H,p_value
1,its_class,54.241561,1.665692e-12
0,locale,118.621422,6.383804e-22
2,mitotype,84.430028,3.980560e-20


In [39]:
# epsilon-squared for Kruskal–Wallis: ε² = (H - k + 1) / (n - k)
def kw_epsilon_squared(H, k, n):
    return (H - k + 1) / (n - k)

n = len(df)

effsizes = []
for col, H in [("locale", 118.621422),
               ("its_class", 54.241561),
               ("mitotype", 84.430028)]:
    k = df[col].nunique()
    eff = kw_epsilon_squared(H, k, n)
    effsizes.append({"factor": col, "k_groups": k, "epsilon_sq": eff})

pd.DataFrame(effsizes)

,factor,k_groups,epsilon_sq
0,locale,9,0.899361
1,its_class,3,0.404973
2,mitotype,2,0.641769


In [41]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

def fit_vs_null(formula):
    m0 = smf.ols("adm2 ~ 1", data=df).fit()
    m1 = smf.ols(formula, data=df).fit()
    comp = sm.stats.anova_lm(m0, m1)
    p_vs_null = comp["Pr(>F)"].iloc[-1]
    return {"model": formula, "adj_R2": m1.rsquared_adj, "p_vs_null": p_vs_null}

models = [
    fit_vs_null("adm2 ~ C(locale)"),
    fit_vs_null("adm2 ~ C(its_class)"),
    fit_vs_null("adm2 ~ C(mitotype)"),
]
pd.DataFrame(models)


,model,adj_R2,p_vs_null
0,adm2 ~ C(locale),0.922151,4.590957e-66
1,adm2 ~ C(its_class),0.359446,1.235572e-13
2,adm2 ~ C(mitotype),0.334013,2.408434e-13


In [42]:
fit_vs_null("adm2 ~ C(locale) + C(its_class) + C(mitotype)")


{'model': 'adm2 ~ C(locale) + C(its_class) + C(mitotype)',
 'adj_R2': np.float64(0.9242073371133026),
 'p_vs_null': np.float64(5.2966544600474045e-64)}

In [43]:
full_model = smf.ols("adm2 ~ C(locale) + C(its_class) + C(mitotype)", data=df).fit()
print(full_model.summary())


                            OLS Regression Results                            
Dep. Variable:                   adm2   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     146.2
Date:                Tue, 14 Oct 2025   Prob (F-statistic):           5.30e-64
Time:                        13:46:42   Log-Likelihood:                 408.31
No. Observations:                 132   AIC:                            -792.6
Df Residuals:                     120   BIC:                            -758.0
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.99